In [6]:
import os
from music21 import *
import numpy as np

In [7]:
# Dataset used : 
# https://www.kaggle.com/zakarii/lofi-hip-hop-midi/metadata

rootdir = "datasets/midi_songs_zackery"

# Holds all the notes of all of our songs!
allSongs = []

for subdirs, dirs, files in os.walk(rootdir):
    for file in files:
        currentSongPath = os.path.join(subdirs, file)
        currentSongParse = converter.parse(currentSongPath)
        currentSong = []
        
        for el in currentSongParse.recurse():   
            if isinstance(el, note.Note):
                currentSong.append(str(el.pitch))
            elif isinstance(el, chord.Chord):
                # try sorting it => sorting it doesnt make a change!
                #currCord = [str(n) for n in el.pitches]
                #currentSong.append('.'.join(sorted(currCord)))
                #currentSong.append('.'.join(str(n) for n in el.pitches))
                currentSong.append('.'.join(str(n) for n in el.normalOrder))
            # For now, all of our rests will be of the duration 1 / 8 
            # in the second part of the project, we will take their actual duration!
            elif type(el) == note.Rest:
                el.duration.quarterLength = 0.5
                currentSong.append(str(el.fullName))
                
        allSongs.append(currentSong)
        

In [8]:
# Padding element is supposed to signify an end to one song, and the beginning of the other
paddingElement = note.Rest()
paddingElement.duration.quarterLength = 16.0

# Currently we are adding a padding element inbetween 2 songs 
# (might work on this in the future, if it gives bad results!)
def flattenSongs(allSongs):
    allNotes = []
    
    for song in allSongs:
        allNotes.append(str(paddingElement.fullName))
        for note in song:
            allNotes.append(note)
        
    return allNotes 
    #maxLenOfSong = max(map(lambda l : len(l), allSongs))
    
    
allNotes = flattenSongs(allSongs)
#print(allNotes)

In [9]:
possibleValues = set(allNotes)
mappingValuesToInt = dict()
mappingIntToValues = dict()
# LTSM works better with int values than with strings, so we need to map out notes (portrayed as strings) to ints
# (we also made a mapIntToVal, so once we need some values with our model, we can return them to their actual representation)
for i, val in enumerate(possibleValues):
    mappingValuesToInt[val] = i
    mappingIntToValues[i] = val

# kako cemo kasnije kad imamo string, pretvoriti u note?

In [10]:
# we will have a list of 20 values for an input, and the output will be a single value
# our model needs to "figure out" what the next note will be using the previous 20 notes!
seqLen = 20

# TODO: FUTURE IDEAS
# in the future, we might change test out what diffrence does it make when we use a diffrent seqLen
# for instance, seqLen = [1, 5, 10, 20, 100, ...]

# inputs and outputs for our model
X = []
Y = []    
    
for i in range(seqLen, len(allNotes)):
    
    inputValues = allNotes[i - seqLen : i]
    outputValues = allNotes[i]
    
    X.append([mappingValuesToInt[note]  for note in inputValues]);
    Y.append(mappingValuesToInt[outputValues]);

# Why do we need the 1 at the end??
X = np.reshape(X, (len(X), seqLen, 1))
# need to add normalization for X
print(X)

[[205 242 256 ... 242 256 205]
 [242 256 235 ... 256 205 138]
 [256 235 256 ... 205 138 256]
 ...
 [ 14 110 229 ... 252 252 252]
 [110 229 256 ... 252 252 122]
 [229 256 205 ... 252 122 122]]
